In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1732380342473_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

In [3]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW api_df 
USING csv 
OPTIONS (path 's3://project-buck3/TRUSTED/api_db/cleaned_data.csv', header 'true', inferSchema 'true');

"""
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [4]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW local_df 
USING csv 
OPTIONS (path 's3://project-buck3/TRUSTED/local_db/cleaned_data.csv', header 'true', inferSchema 'true');
"""
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [9]:
def print_df_columns(view_name, df_name):
    print(f'{df_name} dataframe columns:')
    columns = spark.sql(f"DESCRIBE {view_name}").collect()
    for column in columns:
        print(f' - {column[0]}')
    print()

print_df_columns("api_df", "Covid API Data")
print_df_columns("local_df", "Covid Local Data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Covid API Data dataframe columns:
 - fecha_reporte
 - departamento_id
 - departamento_nombre
 - municipio_id
 - municipio_nombre
 - edad
 - sexo
 - recuperado
 - fecha_inicio_sintomas
 - fecha_diagnostico
 - fecha_recuperacion
 - fecha_muerte

Covid Local Data dataframe columns:
 - fecha_reporte
 - pruebas_realizadas
 - personas_vacunadas
 - ocupacion_uci
 - tasa_transmision
 - medidas_control

In [10]:
count_query = spark.sql("SELECT COUNT(*) FROM api_df")
count_result = count_query.collect()[0][0]
print(f"El número total de filas en api_df es: {count_result}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

El n?mero total de filas en api_df es: 973

In [11]:
count_query = spark.sql("SELECT COUNT(*) FROM local_df")
count_result = count_query.collect()[0][0]
print(f"El número total de filas en local_df es: {count_result}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

El n?mero total de filas en local_df es: 1413

In [12]:
columns = spark.sql("DESCRIBE api_df").collect()
print("api_df schema:")
for column in columns:
    print(f' - {column[0]}: {column[1]}')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

api_df schema:
 - fecha_reporte: date
 - departamento_id: int
 - departamento_nombre: string
 - municipio_id: int
 - municipio_nombre: string
 - edad: int
 - sexo: string
 - recuperado: string
 - fecha_inicio_sintomas: string
 - fecha_diagnostico: date
 - fecha_recuperacion: string
 - fecha_muerte: string

In [13]:
columns = spark.sql("DESCRIBE local_df").collect()
print("local_df schema:")
for column in columns:
    print(f' - {column[0]}: {column[1]}')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

local_df schema:
 - fecha_reporte: date
 - pruebas_realizadas: int
 - personas_vacunadas: int
 - ocupacion_uci: double
 - tasa_transmision: double
 - medidas_control: string

In [14]:
spark.sql("SELECT * FROM api_df LIMIT 20").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+---------------+-------------------+------------+----------------+----+----+----------+---------------------+-----------------+------------------+------------+
|fecha_reporte|departamento_id|departamento_nombre|municipio_id|municipio_nombre|edad|sexo|recuperado|fecha_inicio_sintomas|fecha_diagnostico|fecha_recuperacion|fecha_muerte|
+-------------+---------------+-------------------+------------+----------------+----+----+----------+---------------------+-----------------+------------------+------------+
|   2020-07-18|             44|            GUAJIRA|       44001|        RIOHACHA|  38|   F|Recuperado|           2020-06-27|       2020-07-11|        2020-07-27|         N/A|
|   2021-01-07|             76|              VALLE|       76001|            CALI|  67|   M|Recuperado|           2020-12-27|       2021-01-05|        2021-02-26|         N/A|
|   2020-07-09|             11|             BOGOTA|       11001|          BOGOTA|  50|   F|Recuperado|           2020-06-29| 

In [15]:
spark.sql("SELECT * FROM local_df LIMIT 20").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+------------------+-------------+----------------+--------------------+
|fecha_reporte|pruebas_realizadas|personas_vacunadas|ocupacion_uci|tasa_transmision|     medidas_control|
+-------------+------------------+------------------+-------------+----------------+--------------------+
|   2020-10-13|               458|                69|        96.69|            2.05|   Sin restricciones|
|   2021-06-18|               264|               130|        87.87|            0.93| Cierre de fronteras|
|   2021-10-13|               678|               751|        77.06|            1.97|   Sin restricciones|
|   2022-06-05|               886|               163|        71.11|            2.05| Cuarentena estricta|
|   2022-09-12|               566|               732|        55.93|            1.78|Restricción nocturna|
|   2023-09-24|               653|               859|        92.79|            1.51| Cierre de fronteras|
|   2023-10-20|               420|            

In [29]:
trusted_dir = "s3://project-buck3/REFINED/spark/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# Crear una vista temporal casos_por_ubicacion con SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW casos_por_ubicacion AS
SELECT departamento_nombre, municipio_nombre, COUNT(*) as count
FROM api_df
GROUP BY departamento_nombre, municipio_nombre
ORDER BY count DESC
""")
# Ejecutar la consulta para mostrar los resultados
spark.sql("SELECT * FROM casos_por_ubicacion LIMIT 20").show()
# Guardar los resultados en un archivo CSV
spark.sql("SELECT * FROM casos_por_ubicacion").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "casos_por_ubicacion")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+-----+
|departamento_nombre|municipio_nombre|count|
+-------------------+----------------+-----+
|             BOGOTA|          BOGOTA|  174|
|              VALLE|            CALI|  164|
|          ANTIOQUIA|        MEDELLIN|   77|
|          ANTIOQUIA|           BELLO|   63|
|          CARTAGENA|       CARTAGENA|   51|
|       BARRANQUILLA|    BARRANQUILLA|   51|
|          SANTANDER| BARRANCABERMEJA|   43|
|          ANTIOQUIA|          ITAGUI|   30|
|          SANTANDER|     BUCARAMANGA|   22|
|             TOLIMA|          IBAGUE|   22|
|          SANTANDER|   FLORIDABLANCA|   21|
|              SUCRE|       SINCELEJO|   20|
|               META|   VILLAVICENCIO|   17|
|          SANTANDER|     PIEDECUESTA|   12|
|     STA MARTA D.E.|     SANTA MARTA|   12|
|             CALDAS|       MANIZALES|   10|
|              CAUCA|         POPAYAN|    9|
|            CORDOBA|        MONTERIA|    9|
|          ANTIOQUIA|        RIONEGRO|    6|
|         

In [31]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW df_converted AS
SELECT *, to_date(fecha_reporte, 'yyyy-MM-dd') as fecha_reporte_converted
FROM api_df
""")
# Crear una vista temporal casos_por_dia con SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW casos_por_dia AS
SELECT fecha_reporte_converted AS fecha_reporte, COUNT(*) as count
FROM df_converted
GROUP BY fecha_reporte_converted
ORDER BY count DESC
""")
# Ejecutar la consulta para mostrar los resultados
spark.sql("SELECT * FROM casos_por_dia LIMIT 20").show()
# Guardar los resultados en un archivo CSV
spark.sql("SELECT * FROM casos_por_dia").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "/casos_por_dia")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|fecha_reporte|count|
+-------------+-----+
|   2020-09-03|  180|
|   2020-07-18|  134|
|   2020-06-25|   96|
|   2021-01-07|   70|
|   2020-11-08|   61|
|   2021-01-14|   60|
|   2020-07-25|   54|
|   2020-08-05|   49|
|   2020-09-23|   45|
|   2020-12-24|   30|
|   2020-08-13|   30|
|   2020-10-23|   29|
|   2021-01-13|   24|
|   2020-12-08|   21|
|   2020-07-09|   18|
|   2020-09-13|   15|
|   2021-01-12|   13|
|   2020-10-03|   12|
|   2020-08-17|   11|
|   2020-08-25|   10|
+-------------+-----+

In [32]:
# Crear una vista temporal casos_por_edad con SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW casos_por_edad AS
SELECT edad, COUNT(*) as count
FROM api_df
GROUP BY edad
ORDER BY count DESC
""")
# Ejecutar la consulta para mostrar los resultados
spark.sql("SELECT * FROM casos_por_edad LIMIT 20").show()
# Guardar los resultados en un archivo CSV
spark.sql("SELECT * FROM casos_por_edad").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "/casos_por_edad")
# Crear una vista temporal casos_por_sexo con SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW casos_por_sexo AS
SELECT sexo, COUNT(*) as count
FROM api_df
GROUP BY sexo
""")
# Ejecutar la consulta para mostrar los resultados
spark.sql("SELECT * FROM casos_por_sexo").show()
# Guardar los resultados en un archivo CSV
spark.sql("SELECT * FROM casos_por_sexo").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "/casos_por_sexo")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|edad|count|
+----+-----+
|  33|   37|
|  32|   34|
|  35|   27|
|  30|   27|
|  25|   24|
|  26|   23|
|  40|   23|
|  52|   23|
|  23|   22|
|  22|   22|
|  31|   22|
|  42|   22|
|  34|   21|
|  24|   21|
|  29|   21|
|  38|   21|
|  41|   20|
|  36|   19|
|  37|   19|
|  39|   19|
+----+-----+

+----+-----+
|sexo|count|
+----+-----+
|   M|  540|
|   F|  433|
+----+-----+

In [33]:
# Crear una vista temporal recuperacion_por_sexo con SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW recuperacion_por_sexo AS
SELECT sexo,
       COUNT(CASE WHEN recuperado = 'Recuperado' THEN 1 END) as casos_recuperados,
       COUNT(*) as total_casos,
       (COUNT(CASE WHEN recuperado = 'Recuperado' THEN 1 END) / COUNT(*)) * 100 as porcentaje_recuperacion
FROM api_df
GROUP BY sexo
""")
# Ejecutar la consulta para mostrar los resultados
spark.sql("SELECT * FROM recuperacion_por_sexo").show()
# Guardar los resultados en un archivo CSV
spark.sql("SELECT * FROM recuperacion_por_sexo").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "/recuperacion_por_sexo")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------+-----------+-----------------------+
|sexo|casos_recuperados|total_casos|porcentaje_recuperacion|
+----+-----------------+-----------+-----------------------+
|   M|              518|        540|      95.92592592592592|
|   F|              415|        433|      95.84295612009238|
+----+-----------------+-----------+-----------------------+

In [34]:
# Crear una vista temporal recuperados con SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW recuperados AS
SELECT *
FROM api_df
WHERE recuperado = 'Recuperado'
""")
# Crear una vista temporal tasa_recuperacion con SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW tasa_recuperacion AS
SELECT departamento_nombre, COUNT(*) as count
FROM recuperados
GROUP BY departamento_nombre
ORDER BY count DESC
""")
# Ejecutar la consulta para mostrar los resultados
spark.sql("SELECT * FROM tasa_recuperacion LIMIT 20").show()
# Guardar los resultados en un archivo CSV
spark.sql("SELECT * FROM tasa_recuperacion").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "/tasa_recuperacion_departamento")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|departamento_nombre|count|
+-------------------+-----+
|          ANTIOQUIA|  217|
|             BOGOTA|  166|
|              VALLE|  148|
|          SANTANDER|  101|
|          CARTAGENA|   51|
|       BARRANQUILLA|   51|
|              SUCRE|   25|
|             TOLIMA|   24|
|               META|   16|
|       CUNDINAMARCA|   15|
|             CALDAS|   12|
|          RISARALDA|   12|
|     STA MARTA D.E.|   12|
|            GUAJIRA|   12|
|              CAUCA|   10|
|          ATLANTICO|   10|
|            CORDOBA|   10|
|            BOLIVAR|    8|
|    NORTE SANTANDER|    7|
|              HUILA|    6|
+-------------------+-----+

In [35]:
# Crear una vista temporal mortalidad_municipios con SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW mortalidad_municipios AS
SELECT municipio_nombre,
       COUNT(CASE WHEN fecha_muerte != 'N/A' THEN 1 END) as total_muertes,
       COUNT(*) as total_casos,
       (COUNT(CASE WHEN fecha_muerte != 'N/A' THEN 1 END) / COUNT(*)) * 100 as porcentaje_mortalidad
FROM api_df
GROUP BY municipio_nombre
ORDER BY porcentaje_mortalidad DESC
""")
# Ejecutar la consulta para mostrar los resultados
spark.sql("SELECT * FROM mortalidad_municipios LIMIT 20").show()
# Guardar los resultados en un archivo CSV
spark.sql("SELECT * FROM mortalidad_municipios").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "/mortalidad_municipios")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------+-----------+---------------------+
|municipio_nombre|total_muertes|total_casos|porcentaje_mortalidad|
+----------------+-------------+-----------+---------------------+
|           FUNZA|            1|          1|                100.0|
|        CAUCASIA|            1|          1|                100.0|
|          OBANDO|            1|          1|                100.0|
|         ARMENIA|            1|          2|                 50.0|
|          SOACHA|            1|          6|   16.666666666666664|
|            CALI|           20|        164|   12.195121951219512|
|     PIEDECUESTA|            1|         12|    8.333333333333332|
|   VILLAVICENCIO|            1|         17|     5.88235294117647|
|          BOGOTA|            8|        174|    4.597701149425287|
|     BUCARAMANGA|            1|         22|    4.545454545454546|
|    BARRANQUILLA|            2|         51|   3.9215686274509802|
|        MEDELLIN|            2|         77|   2.5974025974025

In [36]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW df_con_dias AS
SELECT *,
       DATEDIFF(fecha_diagnostico, fecha_inicio_sintomas) AS dias_sintomas_diagnostico,
       DATEDIFF(fecha_recuperacion, fecha_diagnostico) AS dias_diagnostico_recuperacion
FROM api_df
""")
# Crear una vista temporal tiempos_promedio con SQL
spark.sql("""
CREATE OR REPLACE TEMP VIEW tiempos_promedio AS
SELECT 
    AVG(dias_sintomas_diagnostico) AS promedio_dias_sintomas_diagnostico,
    AVG(dias_diagnostico_recuperacion) AS promedio_dias_diagnostico_recuperacion
FROM df_con_dias
""")
# Ejecutar la consulta para mostrar los resultados
spark.sql("SELECT * FROM tiempos_promedio").show()
# Guardar los resultados en un archivo CSV
spark.sql("SELECT * FROM tiempos_promedio").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "/tiempos_promedio")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------+--------------------------------------+
|promedio_dias_sintomas_diagnostico|promedio_dias_diagnostico_recuperacion|
+----------------------------------+--------------------------------------+
|                11.247658688865766|                     23.19506966773848|
+----------------------------------+--------------------------------------+

In [37]:
# Crear una vista temporal con el cálculo de días de recuperación
spark.sql("""
CREATE OR REPLACE TEMP VIEW df_con_dias_recuperacion AS
SELECT *,
       DATEDIFF(fecha_recuperacion, fecha_inicio_sintomas) AS dias_recuperacion
FROM api_df
""")
# Crear una vista temporal con las estadísticas de días de recuperación
spark.sql("""
CREATE OR REPLACE TEMP VIEW dias_recuperacion_estadisticas AS
SELECT municipio_nombre,
       AVG(dias_recuperacion) AS promedio_dias_recuperacion,
       PERCENTILE_APPROX(dias_recuperacion, 0.5) AS mediana_dias_recuperacion
FROM df_con_dias_recuperacion
GROUP BY municipio_nombre
ORDER BY promedio_dias_recuperacion
""")
# Ejecutar la consulta para mostrar los resultados
spark.sql("SELECT * FROM dias_recuperacion_estadisticas LIMIT 20").show()
# Guardar los resultados en un archivo CSV
spark.sql("SELECT * FROM dias_recuperacion_estadisticas").coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "/estadisticas_recuperacion_municipios")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------------+-------------------------+
|   municipio_nombre|promedio_dias_recuperacion|mediana_dias_recuperacion|
+-------------------+--------------------------+-------------------------+
|              FUNZA|                      NULL|                     NULL|
|           CAUCASIA|                      NULL|                     NULL|
|             OBANDO|                      NULL|                     NULL|
|            CAIMITO|                      13.0|                       13|
|        PUERTO RICO|                      15.0|                       15|
|            LA CEJA|                      15.0|                       15|
|SANTA ROSA DE CABAL|                      15.0|                       15|
|            DUITAMA|                      15.0|                       15|
|          ZIPAQUIRA|                      15.0|                       15|
|            PALMIRA|                      15.0|                       15|
|             ARAUCA|    